In [7]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install apscheduler

     -------------------------------------- 59.3/59.3 kB 629.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import pymongo
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime, timedelta

In [2]:
# Initialize MongoDB connection
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["stock_data"]
collection = db["icici_bank"]

In [3]:
def fetch_and_store_data():
    now = datetime.now()
    start_time = now.replace(hour=11, minute=15, second=0, microsecond=0)
    end_time = now.replace(hour=14, minute=15, second=0, microsecond=0)

    if start_time <= now <= end_time:
        ticker = "ICICIBANK.NS"
        icici_bank = yf.Ticker(ticker)
        data = icici_bank.history(period='1d')
        current_time = now.strftime("%Y-%m-%d %H:%M:%S")

        # Store the data in MongoDB
        data_dict = {
            "timestamp": current_time,
            "data": data.to_dict(orient='split')
        }
        collection.insert_one(data_dict)
        print(f"Data saved for {current_time}")

In [4]:
scheduler = BackgroundScheduler()
scheduler.add_job(fetch_and_store_data, 'interval', minutes=15)
scheduler.start()

In [ ]:
try:
    print("Press Ctrl+C to stop the program.")
    while True:
        pass
except KeyboardInterrupt:
    scheduler.shutdown()
    print("Program stopped.")